# Student Analytics Multi-Agent System

Welcome to this interactive demonstration of Praval's multi-agent architecture!

## What You'll Build

A team of **5 specialized agents** that self-organize to answer questions about student performance data:

- 📚 **Schema Expert** - Understands database structure
- 🔨 **Query Builder** - Translates questions to SQL
- ⚙️ **Data Executor** - Runs queries safely
- 📊 **Analyst** - Finds patterns and insights
- 📝 **Reporter** - Formats results for users

## The Praval Way

**No central coordinator!** Each agent:
- Responds to specific message types
- Does ONE thing well
- Broadcasts results for others
- The pipeline emerges naturally through self-organization

Let's get started! 🚀

## Setup

First, let's import Praval and set up our environment.

In [15]:
import sys
import os
from pathlib import Path

# Add parent directory to path
parent_dir = Path(os.getcwd()).parent / 'src'
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

# Import Praval
from praval import agent, start_agents, broadcast

# Import other libraries
import sqlite3
import random
from datetime import datetime, timedelta

print("✅ All imports successful!")

✅ All imports successful!


## Database Setup

Let's create a SQLite database with realistic student performance data.

**Schema:**
- `students`: id, name, grade (6-10), section (A-D)
- `scores`: student_id, subject, score (0-100), test_date

**Subjects:** Mathematics, Science, English, Hindi, Social Studies

In [16]:
def create_student_database(db_path="students.db", num_students_per_section=18):
    """Create and populate the student database."""
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Create tables
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS students (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            grade INTEGER NOT NULL,
            section TEXT NOT NULL
        )
    """)
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS scores (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            student_id INTEGER NOT NULL,
            subject TEXT NOT NULL,
            score INTEGER NOT NULL,
            test_date TEXT NOT NULL,
            FOREIGN KEY (student_id) REFERENCES students(id)
        )
    """)
    
    # Clear existing data
    cursor.execute("DELETE FROM scores")
    cursor.execute("DELETE FROM students")
    
    # Indian names
    first_names = [
        "Aarav", "Vivaan", "Aditya", "Vihaan", "Arjun", "Sai", "Arnav", "Krishna",
        "Ishaan", "Aadhya", "Ananya", "Pari", "Anika", "Diya", "Ishita", "Navya",
        "Saanvi", "Sara", "Priya", "Riya", "Kiara", "Avni", "Kavya", "Myra", "Aditi"
    ]
    
    last_names = [
        "Sharma", "Verma", "Kumar", "Singh", "Patel", "Gupta", "Reddy", "Rao",
        "Iyer", "Nair", "Joshi", "Desai", "Mishra"
    ]
    
    grades = [6, 7, 8, 9, 10]
    sections = ["A", "B", "C", "D"]
    subjects = ["Mathematics", "Science", "English", "Hindi", "Social Studies"]
    
    # Generate students
    student_id = 1
    students_data = []
    
    for grade in grades:
        for section in sections:
            for _ in range(num_students_per_section):
                name = f"{random.choice(first_names)} {random.choice(last_names)}"
                students_data.append((student_id, name, grade, section))
                student_id += 1
    
    cursor.executemany(
        "INSERT INTO students (id, name, grade, section) VALUES (?, ?, ?, ?)",
        students_data
    )
    
    # Generate scores
    scores_data = []
    base_date = datetime.now() - timedelta(days=90)
    
    for student_id, _, grade, _ in students_data:
        for subject in subjects:
            # Grade-appropriate scoring (higher grades score better)
            base_score = 50 + (grade - 6) * 5
            
            # Subject variance
            if subject == "Mathematics":
                score = base_score + random.randint(-15, 20)
            elif subject == "Science":
                score = base_score + random.randint(-10, 20)
            else:
                score = base_score + random.randint(-10, 15)
            
            score = max(0, min(100, score))
            test_date = (base_date + timedelta(days=random.randint(0, 30))).strftime("%Y-%m-%d")
            scores_data.append((student_id, subject, score, test_date))
    
    cursor.executemany(
        "INSERT INTO scores (student_id, subject, score, test_date) VALUES (?, ?, ?, ?)",
        scores_data
    )
    
    conn.commit()
    conn.close()
    
    print(f"✅ Created {len(students_data)} students")
    print(f"✅ Created {len(scores_data)} test scores")
    print(f"💾 Database: {db_path}")
    
    return db_path

# Create the database
DB_PATH = create_student_database()

✅ Created 360 students
✅ Created 1800 test scores
💾 Database: students.db


## Database Schema Reference

All agents will use this schema information.

In [17]:
DATABASE_SCHEMA = """
Table: students
- id (INTEGER): Unique student identifier
- name (TEXT): Student's full name
- grade (INTEGER): Grade level (6-10)
- section (TEXT): Section (A, B, C, D)

Table: scores
- student_id (INTEGER): References students(id)
- subject (TEXT): Mathematics, Science, English, Hindi, Social Studies
- score (INTEGER): Test score (0-100)
- test_date (TEXT): Date of test (YYYY-MM-DD)
"""

print("Database Schema:")
print(DATABASE_SCHEMA)

Database Schema:

Table: students
- id (INTEGER): Unique student identifier
- name (TEXT): Student's full name
- grade (INTEGER): Grade level (6-10)
- section (TEXT): Section (A, B, C, D)

Table: scores
- student_id (INTEGER): References students(id)
- subject (TEXT): Mathematics, Science, English, Hindi, Social Studies
- score (INTEGER): Test score (0-100)
- test_date (TEXT): Date of test (YYYY-MM-DD)



---

# The Multi-Agent System

Now let's build our specialized agents. Each agent responds to specific messages and broadcasts its results.

## Agent 1: Schema Expert 📚

Responds to user queries and provides database context.

In [18]:
@agent("schema_expert", channel="analytics", responds_to=["user_query"])
def schema_agent(spore):
    """
    I understand the database structure and provide schema context
    when someone asks a question.
    """
    question = spore.knowledge.get("question")
    db_path = spore.knowledge.get("db_path")
    
    print(f"📚 Schema Expert: Analyzing question...")
    print(f"📚 Question: '{question}'\n")
    
    # Broadcast schema for query builder
    broadcast({
        "type": "build_query",
        "question": question,
        "schema": DATABASE_SCHEMA,
        "db_path": db_path
    }, channel="analytics")
    
    return {"schema": DATABASE_SCHEMA}

print("✅ Schema Expert agent created")

✅ Schema Expert agent created


## Agent 2: Query Builder 🔨

Translates natural language questions into SQL queries.

In [5]:
@agent("query_builder", channel="analytics", responds_to=["build_query"])
def query_builder_agent(spore):
    """
    I translate natural language questions into SQL queries
    based on the database schema.
    """
    question = spore.knowledge.get("question").lower()
    db_path = spore.knowledge.get("db_path")
    
    print(f"🔨 Query Builder: Translating to SQL...")
    
    # Pattern matching for common queries
    # In production, use an LLM for this
    
    if "average" in question and "mathematics" in question:
        query = """
            SELECT s.grade, AVG(sc.score) as avg_score
            FROM students s
            JOIN scores sc ON s.id = sc.student_id
            WHERE sc.subject = 'Mathematics'
            GROUP BY s.grade
            ORDER BY s.grade
        """
        query_type = "grade_average_math"
    
    elif "top" in question:
        limit = 10
        if "5" in question: limit = 5
        elif "20" in question: limit = 20
        
        query = f"""
            SELECT s.name, s.grade, s.section, AVG(sc.score) as avg_score
            FROM students s
            JOIN scores sc ON s.id = sc.student_id
            GROUP BY s.id
            ORDER BY avg_score DESC
            LIMIT {limit}
        """
        query_type = "top_students"
    
    elif "section" in question:
        query = """
            SELECT s.grade, s.section, AVG(sc.score) as avg_score, COUNT(DISTINCT s.id) as students
            FROM students s
            JOIN scores sc ON s.id = sc.student_id
            GROUP BY s.grade, s.section
            ORDER BY s.grade, s.section
        """
        query_type = "section_performance"
    
    elif "subject" in question:
        query = """
            SELECT sc.subject, AVG(sc.score) as avg_score,
                   MIN(sc.score) as min_score, MAX(sc.score) as max_score
            FROM scores sc
            GROUP BY sc.subject
            ORDER BY avg_score DESC
        """
        query_type = "subject_performance"
    
    elif "failing" in question or "below 40" in question:
        query = """
            SELECT s.name, s.grade, s.section, sc.subject, sc.score
            FROM students s
            JOIN scores sc ON s.id = sc.student_id
            WHERE sc.score < 40
            ORDER BY sc.score ASC
            LIMIT 20
        """
        query_type = "failing_students"
    
    else:
        query = """
            SELECT
                COUNT(DISTINCT s.id) as total_students,
                AVG(sc.score) as overall_avg,
                MIN(sc.score) as lowest_score,
                MAX(sc.score) as highest_score
            FROM students s
            JOIN scores sc ON s.id = sc.student_id
        """
        query_type = "overall_stats"
    
    print(f"🔨 Query type: {query_type}\n")
    
    broadcast({
        "type": "execute_query",
        "query": query,
        "query_type": query_type,
        "question": spore.knowledge.get("question"),
        "db_path": db_path
    }, channel="analytics")
    
    return {"query": query}

print("✅ Query Builder agent created")

✅ Query Builder agent created


## Agent 3: Data Executor ⚙️

Executes SQL queries safely and returns results.

In [6]:
@agent("data_executor", channel="analytics", responds_to=["execute_query"])
def executor_agent(spore):
    """
    I execute SQL queries against the database safely
    and return the results.
    """
    query = spore.knowledge.get("query")
    db_path = spore.knowledge.get("db_path")
    query_type = spore.knowledge.get("query_type")
    question = spore.knowledge.get("question")
    
    print(f"⚙️ Data Executor: Running query...")
    
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute(query)
        results = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        conn.close()
        
        print(f"⚙️ Retrieved {len(results)} rows\n")
        
        broadcast({
            "type": "analyze_results",
            "results": results,
            "columns": columns,
            "query_type": query_type,
            "question": question
        }, channel="analytics")
        
        return {"results": results}
    
    except Exception as e:
        print(f"❌ Error: {e}\n")
        return {"error": str(e)}

print("✅ Data Executor agent created")

✅ Data Executor agent created


## Agent 4: Analyst 📊

Analyzes results to find patterns and key insights.

In [7]:
@agent("analyst", channel="analytics", responds_to=["analyze_results"])
def analyst_agent(spore):
    """
    I analyze query results to find patterns, insights,
    and notable trends in the data.
    """
    results = spore.knowledge.get("results")
    columns = spore.knowledge.get("columns")
    query_type = spore.knowledge.get("query_type")
    question = spore.knowledge.get("question")
    
    print(f"📊 Analyst: Finding insights...")
    
    insights = []
    
    if query_type == "grade_average_math":
        for grade, avg in results:
            insights.append(f"Grade {grade}: {avg:.1f}% average")
            if avg < 60:
                insights.append(f"  ⚠️ Grade {grade} needs improvement")
    
    elif query_type == "top_students":
        insights.append(f"Top {len(results)} performers:")
        for i, (name, grade, section, avg) in enumerate(results[:5], 1):
            insights.append(f"  {i}. {name} (Grade {grade}-{section}): {avg:.1f}%")
    
    elif query_type == "section_performance":
        sorted_res = sorted(results, key=lambda x: x[2], reverse=True)
        if sorted_res:
            best = sorted_res[0]
            worst = sorted_res[-1]
            insights.append(f"Best: Grade {best[0]}-{best[1]} ({best[2]:.1f}%)")
            insights.append(f"Needs attention: Grade {worst[0]}-{worst[1]} ({worst[2]:.1f}%)")
    
    elif query_type == "subject_performance":
        insights.append("Subject rankings:")
        for subject, avg, min_s, max_s in results:
            insights.append(f"  {subject}: {avg:.1f}% avg (range: {min_s}-{max_s})")
    
    elif query_type == "failing_students":
        insights.append(f"Found {len(results)} failing scores (< 40%)")
        if len(results) > 0:
            insights.append("Students needing immediate support:")
            for name, grade, section, subject, score in results[:5]:
                insights.append(f"  {name} (Grade {grade}-{section}): {score}% in {subject}")
    
    elif query_type == "overall_stats":
        total, avg, low, high = results[0]
        insights.append(f"Total students: {total}")
        insights.append(f"Overall average: {avg:.1f}%")
        insights.append(f"Score range: {low}-{high}")
    
    print(f"📊 Found {len(insights)} insights\n")
    
    broadcast({
        "type": "generate_report",
        "results": results,
        "columns": columns,
        "insights": insights,
        "question": question
    }, channel="analytics")
    
    return {"insights": insights}

print("✅ Analyst agent created")

✅ Analyst agent created


## Agent 5: Reporter 📝

Formats insights into a clear, readable report.

In [8]:
@agent("reporter", channel="analytics", responds_to=["generate_report"])
def report_agent(spore):
    """
    I format analysis results into clear, readable reports
    for end users.
    """
    insights = spore.knowledge.get("insights")
    question = spore.knowledge.get("question")
    
    print(f"📝 Reporter: Generating report...\n")
    print("=" * 70)
    print("STUDENT ANALYTICS REPORT")
    print("=" * 70)
    print(f"\nQuestion: {question}\n")
    print("KEY FINDINGS:")
    print("-" * 70)
    for insight in insights:
        print(insight)
    print("\n" + "=" * 70)
    print("✅ Analysis complete!\n")
    
    return {"report": "\n".join(insights)}

print("✅ Reporter agent created")

✅ Reporter agent created


---

# Running Analytics Queries

Now let's use our multi-agent system! Each query triggers the self-organizing pipeline.

## Query 1: Math Performance by Grade

In [9]:
import time

start_agents(
    schema_agent,
    query_builder_agent,
    executor_agent,
    analyst_agent,
    report_agent,
    initial_data={
        "type": "user_query",
        "question": "What is the average score by grade for mathematics?",
        "db_path": DB_PATH
    }
)

time.sleep(0.3)  # Allow agents to complete

📚 Schema Expert: Analyzing question...
📚 Question: 'What is the average score by grade for mathematics?'

🔨 Query Builder: Translating to SQL...
🔨 Query type: grade_average_math

⚙️ Data Executor: Running query...
⚙️ Retrieved 5 rows

📊 Analyst: Finding insights...
📊 Found 7 insights

📝 Reporter: Generating report...

STUDENT ANALYTICS REPORT

Question: What is the average score by grade for mathematics?

KEY FINDINGS:
----------------------------------------------------------------------
Grade 6: 52.6% average
  ⚠️ Grade 6 needs improvement
Grade 7: 58.6% average
  ⚠️ Grade 7 needs improvement
Grade 8: 60.3% average
Grade 9: 67.5% average
Grade 10: 72.4% average

✅ Analysis complete!



## Query 2: Top Performing Students

In [10]:
start_agents(
    schema_agent,
    query_builder_agent,
    executor_agent,
    analyst_agent,
    report_agent,
    initial_data={
        "type": "user_query",
        "question": "Show me the top 10 students by overall performance",
        "db_path": DB_PATH
    }
)

time.sleep(0.3)

📚 Schema Expert: Analyzing question...
📚 Question: 'Show me the top 10 students by overall performance'

🔨 Query Builder: Translating to SQL...
🔨 Query type: top_students

⚙️ Data Executor: Running query...
⚙️ Retrieved 10 rows

📊 Analyst: Finding insights...
📊 Found 6 insights

📝 Reporter: Generating report...

STUDENT ANALYTICS REPORT

Question: Show me the top 10 students by overall performance

KEY FINDINGS:
----------------------------------------------------------------------
Top 10 performers:
  1. Ishaan Desai (Grade 10-B): 81.2%
  2. Aadhya Kumar (Grade 10-B): 80.0%
  3. Ananya Desai (Grade 10-B): 79.8%
  4. Saanvi Singh (Grade 10-A): 79.6%
  5. Myra Rao (Grade 10-C): 79.0%

✅ Analysis complete!



## Query 3: Section Performance Comparison

In [11]:
start_agents(
    schema_agent,
    query_builder_agent,
    executor_agent,
    analyst_agent,
    report_agent,
    initial_data={
        "type": "user_query",
        "question": "What is the performance of each section?",
        "db_path": DB_PATH
    }
)

time.sleep(0.3)

📚 Schema Expert: Analyzing question...
📚 Question: 'What is the performance of each section?'

🔨 Query Builder: Translating to SQL...
🔨 Query type: section_performance

⚙️ Data Executor: Running query...
⚙️ Retrieved 20 rows

📊 Analyst: Finding insights...
📊 Found 2 insights

📝 Reporter: Generating report...

STUDENT ANALYTICS REPORT

Question: What is the performance of each section?

KEY FINDINGS:
----------------------------------------------------------------------
Best: Grade 10-B (73.9%)
Needs attention: Grade 6-C (52.4%)

✅ Analysis complete!



## Query 4: Subject Performance Analysis

In [12]:
start_agents(
    schema_agent,
    query_builder_agent,
    executor_agent,
    analyst_agent,
    report_agent,
    initial_data={
        "type": "user_query",
        "question": "How do different subjects compare in performance?",
        "db_path": DB_PATH
    }
)

time.sleep(0.3)

📚 Schema Expert: Analyzing question...
📚 Question: 'How do different subjects compare in performance?'

🔨 Query Builder: Translating to SQL...
🔨 Query type: subject_performance

⚙️ Data Executor: Running query...
⚙️ Retrieved 5 rows

📊 Analyst: Finding insights...
📊 Found 6 insights

📝 Reporter: Generating report...

STUDENT ANALYTICS REPORT

Question: How do different subjects compare in performance?

KEY FINDINGS:
----------------------------------------------------------------------
Subject rankings:
  Science: 64.7% avg (range: 41-90)
  English: 63.2% avg (range: 40-85)
  Social Studies: 63.1% avg (range: 40-85)
  Mathematics: 62.3% avg (range: 35-90)
  Hindi: 62.2% avg (range: 40-85)

✅ Analysis complete!



## Query 5: Students Needing Help

In [13]:
start_agents(
    schema_agent,
    query_builder_agent,
    executor_agent,
    analyst_agent,
    report_agent,
    initial_data={
        "type": "user_query",
        "question": "Which students are failing (scoring below 40)?",
        "db_path": DB_PATH
    }
)

time.sleep(0.3)

📚 Schema Expert: Analyzing question...
📚 Question: 'Which students are failing (scoring below 40)?'

🔨 Query Builder: Translating to SQL...
🔨 Query type: failing_students

⚙️ Data Executor: Running query...
⚙️ Retrieved 11 rows

📊 Analyst: Finding insights...
📊 Found 7 insights

📝 Reporter: Generating report...

STUDENT ANALYTICS REPORT

Question: Which students are failing (scoring below 40)?

KEY FINDINGS:
----------------------------------------------------------------------
Found 11 failing scores (< 40%)
Students needing immediate support:
  Ishaan Reddy (Grade 6-B): 35% in Mathematics
  Riya Iyer (Grade 6-B): 35% in Mathematics
  Aditya Joshi (Grade 6-B): 36% in Mathematics
  Krishna Verma (Grade 6-B): 36% in Mathematics
  Anika Rao (Grade 6-B): 36% in Mathematics

✅ Analysis complete!



---

# Try Your Own Queries!

Modify the cell below to ask your own questions. The agents will self-organize to answer them.

**Supported query patterns:**
- Questions with "average" + "mathematics" → Math scores by grade
- Questions with "top" + number → Top N students
- Questions with "section" → Section-wise performance
- Questions with "subject" → Subject comparison
- Questions with "failing" or "below 40" → Students needing help
- Other questions → Overall statistics

In [14]:
# Try your own question!
YOUR_QUESTION = "Which is overall the best class in terms of academic performance?"  # Modify this

start_agents(
    schema_agent,
    query_builder_agent,
    executor_agent,
    analyst_agent,
    report_agent,
    initial_data={
        "type": "user_query",
        "question": YOUR_QUESTION,
        "db_path": DB_PATH
    }
)

time.sleep(0.3)

📚 Schema Expert: Analyzing question...
📚 Question: 'Which is overall the best class in terms of academic performance?'

🔨 Query Builder: Translating to SQL...
🔨 Query type: overall_stats

⚙️ Data Executor: Running query...
⚙️ Retrieved 1 rows

📊 Analyst: Finding insights...
📊 Found 3 insights

📝 Reporter: Generating report...

STUDENT ANALYTICS REPORT

Question: Which is overall the best class in terms of academic performance?

KEY FINDINGS:
----------------------------------------------------------------------
Total students: 360
Overall average: 63.1%
Score range: 35-90

✅ Analysis complete!



---

# Explore the Database Directly

Want to see the raw data? Run SQL queries directly!

In [ ]:
# Quick database explorer
def query_db(sql):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute(sql)
    results = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    conn.close()
    
    print("Columns:", columns)
    print("\nResults:")
    for row in results[:10]:  # Show first 10
        print(row)
    if len(results) > 10:
        print(f"\n... and {len(results) - 10} more rows")
    return results

# Example: See some students
query_db("SELECT * FROM students LIMIT 10")

In [ ]:
# Example: See some scores
query_db("""
    SELECT s.name, s.grade, sc.subject, sc.score
    FROM students s
    JOIN scores sc ON s.id = sc.student_id
    LIMIT 15
""")

---

# What You've Learned

## Praval Multi-Agent Architecture

✅ **Self-Organization** - No central coordinator! Agents respond to messages  
✅ **Specialization** - Each agent has ONE clear responsibility  
✅ **Message Types** - Agents filter what they respond to  
✅ **Broadcast Pattern** - Results flow naturally through the system  
✅ **Emergent Pipeline** - Complex workflows from simple rules  

## The Flow

```
user_query → Schema Expert
                 ↓
           build_query → Query Builder
                            ↓
                      execute_query → Data Executor
                                          ↓
                                   analyze_results → Analyst
                                                        ↓
                                                generate_report → Reporter
```

## Next Steps

- Add more query patterns to the Query Builder
- Create visualization agents
- Add an LLM-powered query builder
- Build recommendation agents
- Export reports to PDF/Excel

The possibilities are endless when agents collaborate! 🚀